In [12]:
!pip install groq
!pip install -qU langchain_community
!pip install pdfplumber

In [13]:
import os

os.environ["GROQ_API_KEY"] = "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"
from groq import Groq

from langchain_community.document_loaders import PDFPlumberLoader

In [14]:
def extract_text_pdf(pdf_path):
        loader = PDFPlumberLoader(pdf_path)
        docs = loader.load()
        text = ''
        for doc in docs:
            text += doc.page_content # Append extracted text
        return text 


In [15]:
system_prompt = """You are an AI assistant designed to extract structured resume data.
                   Always respond with a strictly valid JSON object. Use `null` for missing values,
                   ensuring compliance with JSON standards. Do not include explanations,
                   comments, or any additional text outside the JSON structure.
                """



In [16]:
human_prompt = """
             **Task:** Extract key information from the following resume text.

            **Resume Text:**
            {context}

            **Instructions:**
            Please extract the following information and format it in a clear structure:

            1. **Contact Information:**
            - Name:
            - Email:
            - Phone Number:
            - Website/Portfolio/LinkedIn:
            - Github Profile:

            2. **Education:**
            - Institution Name:
            - Degree:
            - Graduation Date:

            3. **Experience:**
            - Job Title:
            - Company Name:
            - Location:
            - Dates of Employment:
            - Description:

            5. **Skills:**
            - Skills:

            **Question:**
            Extract this information as a structured and valid JSON object. Use `null` for missing or unavailable valuesDo not include explanations,
                   comments, or any additional text outside the JSON structure.
        """


In [17]:
pdf_path = "/kaggle/input/steve-qroq/Steve Sun - Resume (2).pdf"

In [18]:
context = extract_text_pdf(pdf_path)

In [19]:
client = Groq()
completion = client.chat.completions.create(
    model="llama-3.3-70b-versatile",
    messages=[
        {
            "role": "system",
            "content": system_prompt
        },
        {
            "role": "user",
            "content": human_prompt.format(context= context)
        }
    ],
    temperature=1,
    max_completion_tokens=1024,
    top_p=1,
    stream=True,
    stop=None,
)

for chunk in completion:
    print(chunk.choices[0].delta.content or "", end="")

```
{
  "Contact Information": {
    "Name": "Steve Sun",
    "Email": "stevesun1245@gmail.com",
    "Phone Number": "(123)456789",
    "Website/Portfolio/LinkedIn": "linkedin.com/in/stevesun",
    "Github Profile": null
  },
  "Education": [
    {
      "Institution Name": "Skyline University",
      "Degree": "Master of Science in Data Science",
      "Graduation Date": "May 2020",
      "GPA": "3.81"
    }
  ],
  "Experience": [
    {
      "Job Title": "Data Scientist",
      "Company Name": "DataNova Insights",
      "Location": "Hyderabad",
      "Dates of Employment": "March 2021 - Present",
      "Description": "Developed and deployed machine learning models to optimize customer retention, increasing engagement by 15%. Conducted exploratory data analysis (EDA) and statistical testing to derive actionable business insights. Built scalable ETL pipelines to process large datasets using Python and SQL. Collaborated with cross-functional teams to improve decision-making through pred